In [1]:
import numpy as np
import modin.pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import ray
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2023-05-22 13:14:01,693	INFO worker.py:1616 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.4.0
Dashboard:,http://127.0.0.1:8265


In [3]:
home = 'C:\\Users\\iksri\\Documents\\Data Science\\Projects\\Credit Risk\\Home Loan Level Data - Freddie Mac\\Datasets'

In [4]:
q1_2010 = pd.DataFrame()
q2_2010 = pd.DataFrame()
q3_2010 = pd.DataFrame()
q4_2010 = pd.DataFrame()
q1_2011 = pd.DataFrame()
q2_2011 = pd.DataFrame()
q3_2011 = pd.DataFrame()
q4_2011 = pd.DataFrame()
q1_2012 = pd.DataFrame()
q2_2012 = pd.DataFrame()
q3_2012 = pd.DataFrame()
q4_2012 = pd.DataFrame()
q1_2013 = pd.DataFrame()
q2_2013 = pd.DataFrame()
q3_2013 = pd.DataFrame()
q4_2013 = pd.DataFrame()
q1_2014 = pd.DataFrame()
q2_2014 = pd.DataFrame()
q3_2014 = pd.DataFrame()
q4_2014 = pd.DataFrame()
q1_2015 = pd.DataFrame()
q2_2015 = pd.DataFrame()
q3_2015 = pd.DataFrame()
q4_2015 = pd.DataFrame()
q1_2016 = pd.DataFrame()
q2_2016 = pd.DataFrame()
q3_2016 = pd.DataFrame()
q4_2016 = pd.DataFrame()
q1_2017 = pd.DataFrame()
q2_2017 = pd.DataFrame()
q3_2017 = pd.DataFrame()
q4_2017 = pd.DataFrame()

qdatasets = [q1_2010, 
            q2_2010, 
            q3_2010, 
            q4_2010, 
            q1_2011, 
            q2_2011, 
            q3_2011, 
            q4_2011,
            q1_2012, 
            q2_2012, 
            q3_2012, 
            q4_2012, 
            q1_2013, 
            q2_2013, 
            q3_2013, 
            q4_2013,
            q1_2014, 
            q2_2014, 
            q3_2014, 
            q4_2014, 
            q1_2015, 
            q2_2015, 
            q3_2015, 
            q4_2015,
            q1_2016, 
            q2_2016, 
            q3_2016, 
            q4_2016, 
            q1_2017, 
            q2_2017, 
            q3_2017, 
            q4_2017]

y2010 = pd.DataFrame()
y2011 = pd.DataFrame()
y2012 = pd.DataFrame()
y2013 = pd.DataFrame()
y2014 = pd.DataFrame()
y2015 = pd.DataFrame()
y2016 = pd.DataFrame()
y2017 = pd.DataFrame()

ydatasets = [y2010,
             y2011,
             y2012,
             y2013,
             y2014,
             y2015,
             y2016,
             y2017,]

In [5]:
i = 0
j = 0
for yr in range(2010, 2018):
    for q in range(1, 5):
        qdatasets[j] = pd.read_csv(home + f"\\{yr}_{q}.csv", index_col = 'Loan Sequence Number')
        qdatasets[j].drop('Unnamed: 0', axis = 1, inplace = True)
        j += 1
    ydatasets[i] = pd.concat([qdatasets[j - 1], qdatasets[j - 2], qdatasets[j - 3], qdatasets[j - 4]])
    i += 1

In [6]:
for df in ydatasets:
    
    # removing typos in column names
    df.columns = ['Credit Score', 'First Payment Date', 'First Time Homebuyer Flag',
                  'Maturity Date', 'Metropolitan Statistical Area',
                  'Mortgage Insurance Percentage', 'Number of Units', 'Occupancy Status',
                  'Original Combined Loan-to-Value', 'Original Debt-to-Income Ratio',
                  'Original UPB', 'Original Loan-to-Value', 'Original Interest Rate',
                  'Channel', 'Prepayment Penalty Mortgage Flag', 'Amortization Type',
                  'Property State', 'Property Type', 'Postal Code', 'Loan Purpose',
                  'Original Loan Term', 'Number of Borrowers', 'Seller Name',
                  'Servicer Name', 'Super Conforming Flag',
                  'Pre-Relief Refinance Loan Sequence Number', 'Program Indicator',
                  'Relief Refinance Indicator', 'Property Valuation Method',
                  'Interest Only Indicator', 'Delinquent']
    
    # dropping all HARP loans
    df.drop(df[df['Relief Refinance Indicator'].isnull() == False].index, inplace = True)
    
    # dropping unneeded or repetitve features
    df.drop(['Original Combined Loan-to-Value', 'Metropolitan Statistical Area', 'Mortgage Insurance Percentage', 'Pre-Relief Refinance Loan Sequence Number', 'Program Indicator', 'Relief Refinance Indicator', 'Property Valuation Method', 'Prepayment Penalty Mortgage Flag', 'Amortization Type', 'Seller Name', 'Servicer Name', 'Super Conforming Flag', 'Interest Only Indicator'], axis = 1, inplace = True)
    
    # highlighting non-explicity null values
    df['Credit Score'] = df['Credit Score'].apply(lambda x: np.nan if x == 9999 else x)
    df['First Time Homebuyer Flag'] = df['First Time Homebuyer Flag'].apply(lambda x: np.nan if x == '9' else x)
    df['Number of Units'] = df['Number of Units'].apply(lambda x: np.nan if x == 9 else x)
    df['Occupancy Status'] = df['Occupancy Status'].apply(lambda x: np.nan if str(x).isdigit() else x)
    df['Original Debt-to-Income Ratio'] = df['Original Debt-to-Income Ratio'].apply(lambda x: np.nan if x == 999 else x)
    df['Original Loan-to-Value'] = df['Original Loan-to-Value'].apply(lambda x: np.nan if x == 999 else x)
    df['Channel'] = df['Channel'].apply(lambda x: np.nan if x == '9' else x)
    df['Property Type'] = df['Property Type'].apply(lambda x: np.nan if str(x).isdigit() else x)
    df['Loan Purpose'] = df['Loan Purpose'].apply(lambda x: np.nan if str(x).isdigit() else x)
    df['Number of Borrowers'] = df['Number of Borrowers'].apply(lambda x: np.nan if x == 99 else x)
    
    # dropping all remaining NULL values (< 0.001% of data)
    df.dropna(inplace = True) 
    
    # to avoid name inter-feature conflicts during one-hot-encoding 
    df['Property Type'] = df['Property Type'].apply(lambda x: 'CN' if x == 'CO' else x) # condo
    df['Loan Purpose'] = df['Loan Purpose'].apply(lambda x: 'PNR' if x == 'P' else x) # purchase no refinance
    df['Loan Purpose'] = df['Loan Purpose'].apply(lambda x: 'RCO' if x == 'C' else x) # refinance cash out
    df['Loan Purpose'] = df['Loan Purpose'].apply(lambda x: 'RNC' if x == 'N' else x) # refinance no cash
    
    # split date into year and month
    df['First Payment Year'] = df['First Payment Date'].apply(lambda s: int(str(s)[:4]))
    df['First Payment Month'] = df['First Payment Date'].apply(lambda s: int(str(s)[4:7]))
    df['Maturity Year'] = df['Maturity Date'].apply(lambda s: int(str(s)[:4]))
    df['Maturity Month'] = df['Maturity Date'].apply(lambda s: int(str(s)[4:7]))
    df.drop(['First Payment Date', 'Maturity Date'], axis = 1, inplace = True)

In [7]:
i = 0
for yr in range(2010, 2018):
    ydatasets[i].sample(n = len(ydatasets[i]) // 10, random_state = 42).to_csv(home + f"\\sample_{yr}.csv")
    i += 1

(autoscaler +5m15s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +5m15s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


In [8]:
for i in range(8):
    # identify object features and perform one-hot-encoding
    obj_feats = []
    for col in ydatasets[i].columns:
        if ydatasets[i][col].dtype == 'O':
            obj_feats.append(col)
    ydatasets[i] = pd.get_dummies(ydatasets[i], columns = obj_feats, drop_first = True, prefix = '', prefix_sep = '')

In [9]:
i = 0
for yr in range(2010, 2018):
    ydatasets[i].to_csv(home + f"\\full_{yr}.csv")
    i += 1